In [1]:
var = [1, 4, 2, 5, 3]


In [2]:
for i in map(str, var):
    print(i, type(i))


1 <class 'str'>
4 <class 'str'>
2 <class 'str'>
5 <class 'str'>
3 <class 'str'>


In [3]:
var.append(0)
" ".join(map(str, var))


'1 4 2 5 3 0'

In [4]:
def encodage(ne, nj, j, x, y):
    return j * ne**2 + x * ne + y + 1


def decodage(k, ne):
    y = (k - 1) % ne
    x = ((k - 1 - y) // ne) % ne
    j = (k - 1 - y - x * ne) // (ne * ne)
    return j, x, y


print(encodage(3, 4, 1, 0, 1))
print(decodage(11, 3))


11
(1, 0, 1)


In [5]:
var = [1, 2, 3]


def au_moins_un(variables):
    clause = [str(v) for v in variables]
    clause.append("0")
    return [" ".join(clause)]


au_moins_un(var)


['1 2 3 0']

In [6]:
var = [1, 2, 3]


def au_plus_un(variables):
    clauses = []
    n = len(variables)
    for i in range(n):
        for j in range(i + 1, n):
            clauses.append("-{} -{} 0".format(variables[i], variables[j]))
    return clauses


au_plus_un(var)


['-1 -2 0', '-1 -3 0', '-2 -3 0']

In [7]:
from itertools import combinations

l = [[-i, -j] for i, j in combinations(var, 2)]
print(l)
[" ".join(map(str, clause + [0])) for clause in l]


[[-1, -2], [-1, -3], [-2, -3]]


['-1 -2 0', '-1 -3 0', '-2 -3 0']

In [8]:
def encoderC1(ne, nj):
    clauses = []
    for j in range(nj):
        for x in range(ne):
            # Contrainte "au plus 1" pour chaque équipe x et chaque jour j
            domicile = [encodage(ne, nj, j, x, y) for y in range(ne) if y != x]
            exterieur = [encodage(ne, nj, j, y, x) for y in range(ne) if y != x]
            clauses.extend(au_plus_un(domicile + exterieur))
    return clauses


encoderC1(2, 2)


['-2 -3 0', '-3 -2 0', '-6 -7 0', '-7 -6 0']

In [9]:
def encoderC2(ne, nj):
    clauses = []
    for x in range(ne):
        for y in range(ne):
            if x != y:
                list_match = [encodage(ne, nj, j, x, y) for j in range(nj)]
                clauses.extend(au_moins_un(list_match))
                clauses.extend(au_plus_un(list_match))
    return clauses


print(len(encoderC2(3, 4)))
encoderC2(3, 4)


42


['2 11 20 29 0',
 '-2 -11 0',
 '-2 -20 0',
 '-2 -29 0',
 '-11 -20 0',
 '-11 -29 0',
 '-20 -29 0',
 '3 12 21 30 0',
 '-3 -12 0',
 '-3 -21 0',
 '-3 -30 0',
 '-12 -21 0',
 '-12 -30 0',
 '-21 -30 0',
 '4 13 22 31 0',
 '-4 -13 0',
 '-4 -22 0',
 '-4 -31 0',
 '-13 -22 0',
 '-13 -31 0',
 '-22 -31 0',
 '6 15 24 33 0',
 '-6 -15 0',
 '-6 -24 0',
 '-6 -33 0',
 '-15 -24 0',
 '-15 -33 0',
 '-24 -33 0',
 '7 16 25 34 0',
 '-7 -16 0',
 '-7 -25 0',
 '-7 -34 0',
 '-16 -25 0',
 '-16 -34 0',
 '-25 -34 0',
 '8 17 26 35 0',
 '-8 -17 0',
 '-8 -26 0',
 '-8 -35 0',
 '-17 -26 0',
 '-17 -35 0',
 '-26 -35 0']

In [10]:
def encoder(ne, nj):
    contraintes = encoderC1(ne, nj) + encoderC2(ne, nj)
    with open("contraintes.cnf", "w") as f:
        # Ecrire l'en-tête du fichier DIMACS
        f.write("p cnf {} {}\n".format(ne * ne * nj - 1, len(contraintes)))
        # Ecrire les contraintes
        for contrainte in contraintes:
            f.write(contrainte + "\n")

In [11]:
def decoder(output_file, ne, nj, team_names_file):
    # Ouvrir le fichier contenant les noms d'équipe
    with open(team_names_file, "r") as f:
        team_names = [line.strip() for line in f]

    # Ouvrir le fichier de sortie de glucose
    with open(output_file, "r") as f:
        output_lines = f.readlines()

    # Extraire le modèle s'il existe
    model = output_lines[0].split()

    # Si aucun modèle n'a été trouvé, il n'y a pas de solution
    if model == "UNSAT":
        return "Pas de solution"

    # Cas SAT : initialisation du planning sous forme de dictionnaire
    planning = {j: {"Domicile" : None, "Extérieur": None} for j in range(nj)}

    for s in model:
        s = int(s)
        if s > 0:
            j, x, y = decodage(s, ne)
            if team_names_file != None:
                x = team_names[x]
                y = team_names[y]
            planning[j]["Domicile"] = x
            planning[j]["Extérieur"] = y

    return planning


decoder("output.txt", 3, 6, "teams_names.txt")


{0: {'Domicile': 'Chicago Bulls', 'Extérieur': 'Atlanta Hawks'},
 1: {'Domicile': 'Atlanta Hawks', 'Extérieur': 'Chicago Bulls'},
 2: {'Domicile': 'Atlanta Hawks', 'Extérieur': 'LA Lakers'},
 3: {'Domicile': 'Chicago Bulls', 'Extérieur': 'LA Lakers'},
 4: {'Domicile': 'LA Lakers', 'Extérieur': 'Atlanta Hawks'},
 5: {'Domicile': 'LA Lakers', 'Extérieur': 'Chicago Bulls'}}

In [12]:
planning = decoder("output.txt", 3, 6, "teams_names.txt")
print("{:<10} {:<15} {}".format('Jour','Domicile','Extérieur'))
for jour, matchs in planning.items():
    domicile = matchs["Domicile"]
    exterieur = matchs["Extérieur"]
    print(" {:<8} {:<15} {:<10}".format(jour + 1, domicile, exterieur))

Jour       Domicile        Extérieur
 1        Chicago Bulls   Atlanta Hawks
 2        Atlanta Hawks   Chicago Bulls
 3        Atlanta Hawks   LA Lakers 
 4        Chicago Bulls   LA Lakers 
 5        LA Lakers       Atlanta Hawks
 6        LA Lakers       Chicago Bulls


In [13]:
import pandas as pd
df = pd.DataFrame(planning)

In [14]:
def optimiser_nj(ne_min = 1, ne_max = 4):
    """ Cherche le nombre de jour nj minimal afin de pouvoir planifier tous
        les matchs du championnat. Tourne au plus 10s.
        @param ne_min: int
        @param ne_max: int
    """
    nj_opt = [0] * ( ne_max - ne_min + 1 )
    
    for ne in range( ne_min, ne_max + 1 ):
        t_start = time.time()
        nj = 1
        while time.time() - t_start < 10:
            genere_cnf(ne,nj)
            if decoder('championnat.cnf', ne, nj) != None:
                nj_opt[ ne - ne_min ] = nj
                print(nj_opt)
                break
            nj += 1
            
    return nj_opt

In [15]:
import subprocess
import os

current = os.getcwd()

cmd = "glucose contraintes.cnf output.txt"
cmd = ["wsl", "/mnt/d/Downloads/glucose-4.2.1/sources/simp/glucose_static"]
print(cmd)
subprocess.run(cmd, shell=True, capture_output=True)

['wsl', '/mnt/d/Downloads/glucose-4.2.1/sources/simp/glucose_static']


In [7]:
os.getcwd()

'd:\\~Perso\\Etudes\\DAC\\S2\\IAMSI\\tme04'